In [1]:
################################################################################
'''
THIS SECTION IS CREATED IN ORDER TO SPEED UP THE PROCESS. THE DATA HERE IS LOADED
AHEAD OF TIME SO THAT ACCESSES TO THE NGS DATA IS 1000x FASTER. THE PROGRAM IS
DESIGNED SUCH THAT THIS CELL IS IN THE KERNEL MEMORY OF THE JUPYTER NOTEBOOK

'''
################################################################################
import os
import time
import pandas as pd

# LOAD JUST THE PASS PLAYS #  
plays_df = pd.read_csv('/PATH TO/ pass_plays.csv')
# LOAD JUST THE GAME DATA #  
games_df = pd.read_csv('/PATH TO/games.csv')
# LOAD JUST THE PLAYER DATA #  
player_info_df = pd.read_csv('/PATH TO/players.csv')

# LOAD THE FILE PATH TO THE DATA #  
filepath = "PATH/TO/DATA"


# THESE ARE THE DICTIONARIES WHERE ALL OF THE NGS DATA WILL BE STORED #
dfs = {}
footballdfs = {}

a = time.time()

# LOOK AT ALL THE FILES IN THE FILEPATH THAT HAS NGS DATA THEN READ THE DATAFRAME INTO MEMORY#  
for x in (os.listdir(filepath)):
    if('tracking' not in x): continue
    game_df = pd.read_csv(filepath + '/' + x)
    dfs[x + '--' + 'regular'] = game_df

# GO THROUGH EACH PLAY THEN ADD THE RELEVANT NGS DATA TO THE MEMORY BANK#  
for x in range(len(plays_df)):
    playId = plays_df.iloc[x]['playId']
    gameId = plays_df.iloc[x]['gameId']
    
    # GET THE NAME OF THE NGS FILE #  
    trackingfilename = 'tracking_gameId_' + str(gameId) + '.csv'
    
    # LOAD THE DF THAT WE HAD LOADED EARLIER #  
    gamedf = dfs[trackingfilename + '--' + 'regular']
    
    # GET THE NGS DATA FROM THIS PLAY #  
    spec = gamedf[gamedf['playId'] == playId]
    
    # FIND THE TIME THE BALL WAS SNAPPED #  
    game_df_ball_snap = spec[spec['event'] == 'ball_snap']
    
    # FIND THE TIME THE BALL WAS THROWN/ QB WAS SACKED #  
    game_df_ball_thrown = spec[(spec['event'] == 'pass_forward') | (spec['event'] == 'qb_sack') |  (spec['event'] == 'run') | (spec['event'] == 'pass_shovel') | (spec['event'] == 'qb_strip_sack')]
    
    # SKIP BAD DATA #  
    if(len(spec) == 0 or len(game_df_ball_thrown) == 0): continue
        
    # GET THE FRAME WHERE THE BALL WAS THROWN #      
    frameval = min(game_df_ball_thrown['frame.id'])
    
    # GET THE SMALLEST FRAME WHERE THE EVENT HAPPENED #    
    try: frameval2 = min(game_df_ball_snap['frame.id'])
    except: frameval2 = 0
        
    # GET FRAMES BEFORE THE EVENT HAPPENED #    
    valsBeforeBall = spec[(spec['frame.id'] <= frameval) & (frameval2 <= spec['frame.id'])]
    
    # GET THE BALL DATA #    
    ball = spec[spec['displayName'] == 'football']
    
    # SAVE THE DATA INTO THE CACHE FOR EACH CATEGORY #
    dfs[trackingfilename + '--' + 'BALL+' + str(playId)] = ball
    dfs[trackingfilename + '--' + 'REGULAR+' + str(playId)] = spec
    dfs[trackingfilename + '--' + 'ball_snap+' + str(playId)] = game_df_ball_snap
    dfs[trackingfilename + '--' + 'before_ball_thrown+' + str(playId)] = valsBeforeBall
    
b= time.time()

print(b-a)

99.2293472290039


In [3]:
# FINDS SECOND SMALLEST NUMBER #
def second_smallest(numbers):
    m1, m2 = float('inf'), float('inf')
    for x in numbers:
        if x <= m1:
            m1, m2 = x, m1
        elif x < m2:
            m2 = x
    return m2

In [2]:
import numpy as np

# SAVE POSITIONS FOR OFFENSE AND DEFENSE #
OFFENSIVE_POSITIONS = set(["QB","RB","WR","T","G","C","TE", "UND"])
DEFENSIVE_POSITIONS = set(["MLB", "OLB", "CB", "SS","FS","DT","DE"])

# BASIC DISTANCE FUNCTION #
def dist(p1,p2):
    (x1,y1) = p1 
    (x2,y2) = p2
    return ((x1-x2)**2 + (y1-y2)**2) ** 0.5


# THIS IS THE FUNCTION THAT WE USE TO GENERATE FEATURES FOR EACH DB #
##############
'''
inputa is  := (the position of the player, the nflID of the cb we are creating features for, his number)
pos_dict is := a dictionary of all the other players in the play with the key being of type 'inputa' described
               above and the value is a list of tuples (a,b,c,d) at each frame of the play
               where a = x coord,
                     b = y coord,
                     c = s,
                     d = dir, 
                     
time_interval_num := the frame split window (this isn't used in the basic clustering)
'''
##############
def make_feature_row(inputa,pos_dict,time_interval_num):
    # SPLIT inputa AS ABOVE # 
    (pos,nflid,num) = inputa
    
    # GET THIS PLAYERS LIST OF FEATURES PER FRAME # 
    myself = pos_dict[(pos,nflid,num)]
    
    # BREAKUP SIZE %%NOT USED%%# 
    breakupsize = int(len(myself)/5)
    
    # LOAD YOUR PLAYER INFO# 
    me = (pos,nflid,num)
    
    # FINAL RESULT GETS STORED HERE # 
    res = {}
    a1 = time.time()
    
    # STORE PLAYER PATH IN THE CACHE #
    res["MYPATH"] = myself
    
    ### DISTANCE TO NEAREST OFFENSIVE PLAYER, taken 5 TIMES BEFORE BALL THROWN ###
    for x in range(5):
        myframe = myself[x*breakupsize]
        myx,myy = myframe[0],myframe[1]
        minthing = []
        tempthing = []
        poses = []
        nums = []
        #print("MY COORDS ARE " + str((myx,myy)))
        for k,v in pos_dict.items():
            if(k == me): continue
            if(len(v) ==0): continue
            if(k[0] not in OFFENSIVE_POSITIONS): continue
            try:
                p2 = (v[x][0],v[x][1])
            except: continue
            tempthing.append(p2)
            minthing.append(dist((myx,myy),p2))
            poses.append(k[0])
            nums.append(k[2])
        #print(minthing,poses,nums,tempthing)
        if(len(minthing) == 0): continue
        valval = min(minthing)
        
        minpos = poses[np.argmin(minthing)]
        minnum = nums[np.argmin(minthing)]
        res[str(x) + '-DIS_OFF'] = valval
        res[str(x) + '-POS_OFF'] = minpos
        res[str(x) + '-NUM_OFF'] = minnum
        
    ### VARIANCE OF DISTANCE FROM NEAREST OFFENSIVE PLAYER ####
    ### VARIANCE OF RATIO BETWEEN NEAREST DEFENDERS to the CLOSEST OFFENSIVE PLAYER THROUGH PLAY  ####
    ### MEAN RATIO BETWEEN NEAREST DEFENDERS to the CLOSEST OFFENSIVE PLAYER THROUGH PLAY  ####
    ### RATIO BETWEEN NEAREST DEFENDERS to the CLOSEST OFFENSIVE PLAYER AT SNAP ####
    ### RATIO BETWEEN NEAREST DEFENDERS to the CLOSEST OFFENSIVE PLAYER AT PASS ####
    ### RATIO BETWEEN NEAREST DEFENDERS to the CLOSEST OFFENSIVE PLAYER AT HALFWAY ####
    ### VARIANCE OF RATIO BETWEEN NEAREST DEFENDERS ####
    ### MEAN DIFFERENCE IN DIRECTION BETWEEN THE DEFENDER AND NEAREST OFFENSIVE PLAYER ####
    ### VARIANCE IN DIFFERENCE IN DIRECTION BETWEEN THE DEFENDER AND NEAREST OFFENSIVE PLAYER ####
    dist_to_nearest_off = []
    dir_comp_off = []
    dist_second_nearest_off = []
    nearest_off_coords = []
    for x in range(len(myself)):
        minthing = []
        min_dir = []
        min_coords = []
        for k,v in pos_dict.items():
            if(k == me): continue
            if(len(v) == 0): continue
            if(k[0] not in OFFENSIVE_POSITIONS): continue
            p1 = (myself[x][0],myself[x][1])
            p2 = (v[x][0],v[x][1])
            minthing.append(dist(p1,p2))
            min_coords.append(p2)
            min_dir.append(abs(v[x][3] - myself[x][3]))
        dist_to_nearest_off.append(min(minthing))
        dist_second_nearest_off.append(second_smallest(minthing))
        dir_comp_off.append(min_dir[np.argmin(minthing)])
        nearest_off_coords.append(min_coords[np.argmin(minthing)])
    rats = []
    for x in range(len(dist_to_nearest_off)):
        rat = float(dist_to_nearest_off[x])/float(dist_second_nearest_off[x])
        rats.append(rat)
    res['RAT-type1-MEAN'] = np.mean(rats)
    res['RAT-type1-VAR'] = np.var(rats)
    res['RAT-type1-SNAP'] = rats[0]
    res['RAT-type1-PASS'] = rats[len(rats)-1]
    res['RAT-type1-MID'] = rats[(len(rats)-1)//2]
    res['OFF_VAR'] = np.var(dist_to_nearest_off)
    res['OFF_DIR_MEAN'] = np.mean(dir_comp_off)
    res['OFF_DIR_VAR'] = np.var(dir_comp_off)
    
    ### VARIANCE OF DISTANCE FROM NEAREST DEFENSIVE PLAYER ####
    ### MEAN IN DIFFERENCE IN DIRECTION BETWEEN THE DEFENDER AND NEAREST DEFENSIVE PLAYER  ####
    ### VARIANCE IN DIFFERENCE IN DIRECTION BETWEEN THE DEFENDER AND NEAREST DEFENSIVE PLAYER  ####
    ### VARIANCE OF RATIO BETWEEN THE DISTANCE FROM THE NEAREST OFFENSIVE PLAYER TO THE DB AND THE SECOND NEAREST DEFENDER TO THE OFFENSIVE PLAYER THROUGHOUT THE PLAY  ####
    ### MEAN RATIO BETWEEN THE DISTANCE FROM THE NEAREST OFFENSIVE PLAYER TO THE DB AND THE SECOND NEAREST DEFENDER TO THE OFFENSIVE PLAYER THROUGHOUT THE PLAY  ####
    ### RATIO BETWEEN THE DISTANCE FROM THE NEAREST OFFENSIVE PLAYER TO THE DB AND THE SECOND NEAREST DEFENDER TO THE OFFENSIVE PLAYER THROUGHOUT THE PLAY AT SNAP  ####
    ### RATIO BETWEEN THE DISTANCE FROM THE NEAREST OFFENSIVE PLAYER TO THE DB AND THE SECOND NEAREST DEFENDER TO THE OFFENSIVE PLAYER THROUGHOUT THE PLAY AT PASS ####
    ### RATIO BETWEEN THE DISTANCE FROM THE NEAREST OFFENSIVE PLAYER TO THE DB AND THE SECOND NEAREST DEFENDER TO THE OFFENSIVE PLAYER THROUGHOUT THE PLAY AT HALFWAY ####
    dist_to_nearest_def = []
    dir_comp_def = []
    nearest_def_coords = []
    for x in range(len(myself)):
        minthing = []
        min_dir = []
        min_coords = []
        for k,v in pos_dict.items():
            if(k == me): continue
            if(len(v) == 0): continue
            if(k[0] not in DEFENSIVE_POSITIONS): continue
            p1 = (myself[x][0],myself[x][1])
            p2 = (v[x][0],v[x][1])
            minthing.append(dist(p1,p2))
            min_coords.append(p2)
            min_dir.append(abs(v[x][3] - myself[x][3]))
        dist_to_nearest_def.append(min(minthing))
        dir_comp_def.append(min_dir[np.argmin(minthing)])
        nearest_def_coords.append(min_coords[np.argmin(minthing)])
    res['DEF_VAR'] = np.var(dist_to_nearest_def)
    res['DEF_DIR_MEAN'] = np.mean(dir_comp_def)
    res['DEF_DIR_VAR'] = np.var(dir_comp_def)
    dists_def_off = []
    for i in range(len(nearest_off_coords)):
        d = dist(nearest_off_coords[i],nearest_def_coords[i])
        dists_def_off.append(d)
    newrats = []
    for i in range(len(dists_def_off)):
        rr = float(dist_to_nearest_off[i])/float(dists_def_off[i])
        newrats.append(rr)
    res['RAT-type2-MEAN'] = np.mean(newrats)
    res['RAT-type2-VAR'] = np.var(newrats)
    res['RAT-type2-SNAP'] = newrats[0]
    res['RAT-type2-PASS'] = newrats[len(newrats)-1]
    res['RAT-type2-MID'] = newrats[(len(newrats)-1)//2]
    
    
    ### VARIANCE OF SPEED THROUHOUT ###
    var_speed = np.var([x[2] for x in myself])
    res['SPEED_VAR'] = var_speed
    #print(var_speed)
    
    ### MEAN DISTANCE FROM NEAREST DEFENDER ###
    dist_to_nearest_def_mean = []
    for x in range(len(myself)):
        minthing = []
        for k,v in pos_dict.items():
            if(k == me): continue
            if(len(v) == 0): continue
            if(k[0] not in DEFENSIVE_POSITIONS): continue
            p1 = (myself[x][0],myself[x][1])
            p2 = (v[x][0],v[x][1])
            minthing.append(dist(p1,p2)) 
        dist_to_nearest_def_mean.append(min(minthing))
    res['DEF_MEAN'] = np.mean(dist_to_nearest_def_mean)
    #print(np.mean(dist_to_nearest_def_mean))
    
    ### MEAN DISTANCE FROM NEAREST OFFENSIVE PLAYER ### 
    dist_to_nearest_off_mean = []
    for x in range(len(myself)):
        minthing = []
        for k,v in pos_dict.items():
            if(k == me): continue
            if(len(v) == 0): continue
            if(k[0] not in OFFENSIVE_POSITIONS): continue
            p1 = (myself[x][0],myself[x][1])
            p2 = (v[x][0],v[x][1])
            minthing.append(dist(p1,p2))
            
        dist_to_nearest_off_mean.append(min(minthing))
    res['OFF_MEAN'] = np.mean(dist_to_nearest_off_mean)
    
    
    ### VARIANCE OF OWN MOTION X ###
    var_x = np.var([x[0] for x in myself])
    res['VAR_X'] = var_x
    
    
    ### VARIANCE OF OWN MOTION Y ###
    var_y = np.var([x[1] for x in myself])
    res['VAR_Y'] = var_y
    #print(var_y)
    b1 = time.time()
     
    
    if(len(res.keys()) == 0): print("EMPTY")
    return res

### THE OUTPUT IS GOING HERE ###
to_df = []


### LOOP OVER ALL THE PLAYS ###
for x in range(len(plays_df)):

    a1 = time.time()
    ### GET THE PLAY DATA ###
    play = plays_df.iloc[x]
    gameId = play['gameId']
    playId = play['playId']
    trackingfilename = 'tracking_gameId_' + str(gameId) + '.csv'
    
    ### IGNORE BAD DATA ###
    try: before_ball_thrown = dfs[trackingfilename + '--' + 'before_ball_thrown+' + str(playId)]
    except: continue
        
    ### GET BALL DATA ###
    ball = dfs[trackingfilename + '--' + 'BALL+' + str(playId)]
    
    prev = None
    prevNum = None
    curr = []
    pos_dicts = {}
    a2 = time.time()
    
    ### CREATE THE DATA FORM DESCRIBED IN THE INPUT TO THE FEATURE FUNCTION ###
    for index, row in before_ball_thrown.iterrows():
        if(row['displayName'] == 'football'): continue
        if(row['nflId'] == prev):
            
            curr.append((row['x'],row['y'],row['s'],row['dir']))
        else:
            if(prev != None):
                position = (player_info_df[player_info_df['nflId'] == prev]).iloc[0]['PositionAbbr']
                pos_dicts[(position,prev,prevNum)] = curr
            prev = row['nflId']
            prevNum = row['jerseyNumber']
            curr = []
            curr = [(row['x'],row['y'],row['s'],row['dir'])]
    b2 = time.time()
    
    ### FIND THE CBs AND THEN CREATE FEATURES FOR THEM ###
    for k , v in pos_dicts.items(): 
        if(k[0] == "CB"):
            
            if(len(pos_dicts[k]) == 0): continue
            featurerow = make_feature_row(k,pos_dicts,5)
            featurerow['gameId'] = gameId
            featurerow['playId'] = playId
            featurerow['position'] = k[0]
            featurerow['nflId'] = k[1]
            featurerow['number'] = k[2]
            featurerow['LOS'] = play['LOS']
            featurerow['POSS_DIR'] = play['POSS_DIR']
            to_df.append(featurerow)
        
    b1 = time.time()
    
    print(x, b1-a1)



df = pd.DataFrame(to_df)
df.to_csv("/PATH TO/corners_features_with_ratios.csv")

NameError: name 'plays_df' is not defined